In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Library

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow. keras.models import Sequential
from tensorflow. keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow. keras. optimizers import Adam
from tensorflow. keras.metrics import categorical_crossentropy
from tensorflow. keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from keras.preprocessing import image
import pandas as pd
import seaborn as sns
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter (action='ignore', category=FutureWarning)
%matplotlib inline


In [ ]:
import torch
torch.cuda.is_available()
# check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Data Pre Processing

In [ ]:
train_path = '/content/drive/MyDrive/project/train'
valid_path = '/content/drive/MyDrive/project/val'
test_path = '/content/drive/MyDrive/project/test'

In [ ]:
train_batches = ImageDataGenerator (preprocessing_function=tf.keras.applications.resnet.preprocess_input)\
.flow_from_directory (directory=train_path, target_size=(224,224), classes=['DR', 'No_DR'], batch_size=10)
valid_batches = ImageDataGenerator (preprocessing_function=tf.keras.applications.resnet.preprocess_input)\
.flow_from_directory (directory=valid_path, target_size=(224,224), classes=['DR', 'No_DR'], batch_size=10)
test_batches = ImageDataGenerator (preprocessing_function=tf.keras.applications.resnet.preprocess_input)\
.flow_from_directory (directory=test_path, target_size=(224,224), classes=['DR', 'No_DR'], batch_size=10, shuffle=False)

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
#This function will plot images in the form of a grid with 1 row and 10 columns where images are
def plotImages (images_arr):
    fig, axes= plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages (imgs)
print(labels)

MODEL Building

In [ ]:
ResNet152_model = tf.keras.applications.ResNet152(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
ResNet152_model. summary()

In [ ]:
type (ResNet152_model)

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet152

# Exclude the last layer
model = Model(inputs=ResNet152_model.input, outputs=ResNet152_model.layers[-2].output)


In [ ]:
model. summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Dense

# Add a Dense layer with softmax activation
output_layer = Dense(units=2, activation='softmax')(model.output)

# Create a new model with DenseNet121 base and the new output layer
model = Model(inputs=model.input, outputs=output_layer)

# Display model summary
model.summary()


In [ ]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
hist=model.fit(x=train_batches, validation_data=valid_batches, epochs=100, verbose=2)

Validation

In [ ]:
#Training set Accuracy
train_loss, train_acc = model.evaluate(train_batches)
print('Training accuracy:', train_acc)

In [ ]:
#Validation set Accuracy
val_loss, val_acc = model.evaluate(valid_batches)
print('Validation accuracy:', val_acc)

In [ ]:
model.save('model.keras')

In [ ]:
hist.history #Return Dictionary of history

In [ ]:
#Recording History in json
import json
with open('training_hist.json','w') as f:
  json.dump(hist.history,f)

In [ ]:
print(hist.history.keys())

In [ ]:
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'], color='red', label='Training Loss')
plt.plot(range(1, len(hist.history['val_loss']) + 1), hist.history['val_loss'], color='blue', label='Validation Loss')
plt.xlabel('No. of Epochs')
plt.title('Visualization of Loss Result')
plt.legend()
plt.show()


In [ ]:
plt.plot(range(1, len(hist.history['accuracy']) + 1), hist.history['accuracy'], color='red', label='Training Accuracy')
plt.plot(range(1, len(hist.history['val_accuracy']) + 1), hist.history['val_accuracy'], color='blue', label='Validation Accuracy')
plt.xlabel('No. of Epochs')
plt.title('Visualization of Accuracy Result')
plt.legend()
plt.show()


Predict

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
test_batches.classes

In [ ]:
cm= confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax (predictions, axis=-1))

In [ ]:
test_batches.class_indices

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

# Assuming 'cm' is your confusion matrix
cm_plot_labels = ['DR', 'No_DR']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cm, classes=cm_plot_labels,
                      title='Confusion Matrix')

plt.show()
